<h2 align="center">BERT tutorial: Classify spam vs no spam emails</h2>

In [2]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

     |████████████████████████████████| 4.9 MB 6.1 MB/s 
     |████████████████████████████████| 462 kB 40.2 MB/s 


<h4>Import the dataset (Dataset is taken from kaggle)</h4>

In [4]:
import pandas as pd

df = pd.read_csv("spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [6]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


<h4>Split it into training and test data set</h4>

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'], stratify=df['spam'])

In [8]:
X_train.head(4)

831     U have a secret admirer. REVEAL who thinks U R...
3130    Haha better late than ever, any way I could sw...
2918           Yes. that will be fine. Love you. Be safe.
4010    Ha... Then we must walk to everywhere... Canno...
Name: Message, dtype: object

<h4>Now lets import BERT model and get embeding vectors for few sample statements</h4>

In [9]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [10]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435166 , -0.5132724 , -0.88845706, ..., -0.7474883 ,
        -0.7531471 ,  0.91964483],
       [-0.8720836 , -0.50544   , -0.9444667 , ..., -0.8584748 ,
        -0.71745366,  0.88082993]], dtype=float32)>

<h4>Get embeding vectors for few sample words. Compare them using cosine similarity</h4>

In [11]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.99110895]], dtype=float32)

Values near to 1 means they are similar. 0 means they are very different.
Above you can use comparing "banana" vs "grapes" you get 0.99 similarity as they both are fruits

In [13]:
cosine_similarity([e[0]],[e[3]])

array([[0.8470383]], dtype=float32)

Comparing banana with jeff bezos you still get 0.84 but it is not as close as 0.99 that we got with grapes

In [14]:
cosine_similarity([e[3]],[e[4]])

array([[0.9872036]], dtype=float32)

Jeff bezos and Elon musk are more similar then Jeff bezos and banana as indicated above

<h4>Build Model</h4>

There are two types of models you can build in tensorflow. 

(1) Sequential
(2) Functional

So far we have built sequential model. But below we will build functional model. More information on these two is here: https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057

In [15]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

https://stackoverflow.com/questions/47605558/importerror-failed-to-import-pydot-you-must-install-pydot-and-graphviz-for-py

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [17]:
len(X_train)

4179

In [18]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

<h4>Train the model</h4>

In [19]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 94s 624ms/step - loss: 0.3877 - accuracy: 0.8394
Epoch 2/5
131/131 [==============================] - 82s 626ms/step - loss: 0.2626 - accuracy: 0.8787
Epoch 3/5
131/131 [==============================] - 82s 625ms/step - loss: 0.2207 - accuracy: 0.9076
Epoch 4/5
131/131 [==============================] - 82s 623ms/step - loss: 0.1898 - accuracy: 0.9273
Epoch 5/5
131/131 [==============================] - 82s 623ms/step - loss: 0.1738 - accuracy: 0.9352


In [20]:
model.evaluate(X_test, y_test)

44/44 [==============================] - 29s 639ms/step - loss: 0.1608 - accuracy: 0.9390


[0.1608494073152542, 0.9389806389808655]

<h4>Inference</h4>

In [27]:
reviews = ["Awesome, text me when you're restocked",
 'Ummmmmaah Many many happy returns of d day my dear sweet heart.. HAPPY BIRTHDAY dear',
 'I think steyn surely get one wicket:)',
 'CAN I PLEASE COME UP NOW IMIN TOWN.DONTMATTER IF URGOIN OUTL8R,JUST REALLYNEED 2DOCD.PLEASE DONTPLEASE DONTIGNORE MYCALLS,U NO THECD ISV.IMPORTANT TOME 4 2MORO',
 '2 and half years i missed your friendship:-)',
 "RCT' THNQ Adrian for U text. Rgds Vatian",
 'FreeMsg Hey U, i just got 1 of these video/pic fones, reply WILD to this txt & ill send U my pics, hurry up Im so bored at work xxx (18 150p/rcvd STOP2stop)',
 'Update_Now - 12Mths Half Price Orange line rental: 400mins...Call MobileUpd8 on 08000839402 or call2optout=J5Q',
 'Both :) i shoot big loads so get ready!',
 "I wish! I don't think its gonna snow that much. But it will be more than those flurries we usually get that melt before they hit the ground. Eek! We haven't had snow since &lt;#&gt; before I was even born!"]
 
model.predict(reviews)

array([[0.02552248],
       [0.07943359],
       [0.28482732],
       [0.20263033],
       [0.18409447],
       [0.05519792],
       [0.54239553],
       [0.716975  ],
       [0.34997115],
       [0.11601462]], dtype=float32)

In [36]:
a = [i for i in  model.predict(reviews)]
for  i in range(len(a)):
  if a[i][0] <=0.5:
    print('Review is ==>  ham')
  else:
    print('Review is ==> spam')

Review is ==>  ham
Review is ==>  ham
Review is ==>  ham
Review is ==>  ham
Review is ==>  ham
Review is ==>  ham
Review is ==> spam
Review is ==> spam
Review is ==>  ham
Review is ==>  ham


In [25]:
reviews = []
for i in X_test.sample(10):
  reviews.append(i)

In [26]:
reviews

["Awesome, text me when you're restocked",
 'Ummmmmaah Many many happy returns of d day my dear sweet heart.. HAPPY BIRTHDAY dear',
 'I think steyn surely get one wicket:)',
 'CAN I PLEASE COME UP NOW IMIN TOWN.DONTMATTER IF URGOIN OUTL8R,JUST REALLYNEED 2DOCD.PLEASE DONTPLEASE DONTIGNORE MYCALLS,U NO THECD ISV.IMPORTANT TOME 4 2MORO',
 '2 and half years i missed your friendship:-)',
 "RCT' THNQ Adrian for U text. Rgds Vatian",
 'FreeMsg Hey U, i just got 1 of these video/pic fones, reply WILD to this txt & ill send U my pics, hurry up Im so bored at work xxx (18 150p/rcvd STOP2stop)',
 'Update_Now - 12Mths Half Price Orange line rental: 400mins...Call MobileUpd8 on 08000839402 or call2optout=J5Q',
 'Both :) i shoot big loads so get ready!',
 "I wish! I don't think its gonna snow that much. But it will be more than those flurries we usually get that melt before they hit the ground. Eek! We haven't had snow since &lt;#&gt; before I was even born!"]

In [ ]:
model.predict(reviews)